In [4]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Carregar os dados
df = pd.read_excel('../baseDeDados/dataBaseTratadaDelivery.xlsx')
dfTeste = pd.read_excel('../baseDeDados/dataBaseTeste.xlsx')

# Criando novas colunas no dataset de treinamento
df['order_weekday'] = pd.to_datetime(df['order_purchase_timestamp']).dt.dayofweek
df['is_sao_paulo_estado_comprador'] = (df['customer_state'] == 'sao paulo').astype(int)
df['is_sao_paulo_cidade_comprador'] = (df['customer_city'] == 'sao paulo').astype(int)
df['is_sao_paulo_estado_vendedor'] = (df['seller_state'] == 'SP').astype(int)
df['is_sao_paulo_cidade_vendedor'] = (df['seller_city'] == 'sao paulo').astype(int)
df['is_credit_card'] = (df['payment_type'] == 'credit_card').astype(int)

# Criando novas colunas no dataset de teste
dfTeste['order_weekday'] = pd.to_datetime(dfTeste['order_purchase_timestamp']).dt.dayofweek
dfTeste['is_sao_paulo_estado_comprador'] = (dfTeste['customer_state'] == 'sao paulo').astype(int)
dfTeste['is_sao_paulo_cidade_comprador'] = (dfTeste['customer_city'] == 'sao paulo').astype(int)
dfTeste['is_sao_paulo_estado_vendedor'] = (dfTeste['seller_state'] == 'SP').astype(int)
dfTeste['is_sao_paulo_cidade_vendedor'] = (dfTeste['seller_city'] == 'sao paulo').astype(int)
dfTeste['is_credit_card'] = (dfTeste['payment_type'] == 'credit_card').astype(int)

# Preparar os dados de treinamento para Prophet
train_data = df.copy()

train_prophet_data = train_data[['order_purchase_timestamp', 'delivery_time']].copy()
train_prophet_data.rename(columns={'order_purchase_timestamp': 'ds', 'delivery_time': 'y'}, inplace=True)
train_prophet_data.dropna(inplace=True)
train_prophet_data['y'] = train_prophet_data['y'].apply(lambda x: max(x, 13))
train_prophet_data['y'] = np.log1p(train_prophet_data['y'] - 11)

# Criar e treinar o modelo Prophet com ajustes de sazonalidade
model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.fit(train_prophet_data)

# Preparar os dados de teste para Prophet
test_prophet_data = dfTeste[['order_purchase_timestamp']].copy()
test_prophet_data.rename(columns={'order_purchase_timestamp': 'ds'}, inplace=True)

# Fazer previsões nos dados de teste
forecast_test = model.predict(test_prophet_data)
forecast_test['yhat'] = np.expm1(forecast_test['yhat']) + 11

# Adicionar previsões ao dataset de teste
dfTeste['previsoes'] = forecast_test['yhat']

# Interligar com modelo linear múltiplo
features = [
    'customer_zip_code_prefix', 'is_sao_paulo_estado_comprador', 'is_sao_paulo_cidade_comprador', 
    'is_sao_paulo_estado_vendedor', 'is_sao_paulo_cidade_vendedor', 'is_credit_card', 
    'Distancia', 'Latitude_Vendedor', 'Latitude_Comprador', 'Longitude_Comprador', 'Longitude_Vendedor'
]
X_train = train_data[features]
y_train = train_data['delivery_time']
X_test = dfTeste[features]

# Modelo Linear Múltiplo
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
dfTeste['linear_pred'] = linear_model.predict(X_test)

# Mostrar previsões
dfTeste[['previsoes', 'linear_pred']].head()


20:07:38 - cmdstanpy - INFO - Chain [1] start processing
20:07:45 - cmdstanpy - INFO - Chain [1] done processing


,previsoes,linear_pred
0,13.822557,11.473356
1,13.713156,11.558386
2,13.614164,11.435823
3,13.648888,12.402332
4,13.576637,12.527124


In [5]:
# Enviando para um dataFrame do Kaggle
dfKaggle = dfTeste[['order_id', 'linear_pred']]

# Salvar os resultados
dfKaggle.to_csv('../resultadoPrevisoes/previsaoProphetLinearMultipla.csv', index=False)

In [6]:
dfKaggle

,order_id,linear_pred
0,cee3292f46ede6ea1dfabfcb200fcf47,11.473356
1,50dca53ca33b739bef09e7933e8b380e,11.558386
2,8087ec71e393d4dc6fc48041fe63cd51,11.435823
3,e6b6557ce111de79b31cc857f20ba212,12.402332
4,0b09c5e4c2512f627190ac55a78c35a3,12.527124
...,...,...
28147,81cedc4d2132eb6f512354166e51b187,11.527259
28148,95811675a89b66659996da5285a021db,12.656580
28149,25bd2ad9d282f59e4e6a048ef9bf214f,14.234255
28150,9f9ac312604229686c04260955495c70,11.244646
